In [1]:
import numpy as np
import pandas as pd

In [4]:
from IPython.core.magic import register_line_magic

@register_line_magic
def C(line):
    from IPython.core.getipython import get_ipython
    from fnmatch import fnmatch

    line = line.strip()
    idx_space = line.index(' ')
    space_num = line[:idx_space]
    if space_num.isdecimal():
        space_num = int(space_num)
        line = line[idx_space:]
    else:
        space_num = 5

    output_dict = {}
    cmds = line.split(';')
    for cmd in cmds:
        cmd = cmd.strip()
        output_dict[cmd] = repr(eval(cmd)).split("\n")

    str_maxlen_in_cols = [max(len(cmd), len(max(data, key=len))) for cmd, data in output_dict.items()]
    data_row_max = max([len(v) for v in output_dict.values()])

    out_lines = [""]*(data_row_max+2)

    space=''
    for i, (cmd, data) in enumerate(output_dict.items()):
        w = str_maxlen_in_cols[i]

        out_lines[0]+=space+f'{cmd:^{w}}'
        out_lines[1]+=space+"-"*w
        for j, d in enumerate(data, 2):
            out_lines[j]+=space+f'{d:{w}}'

        if len(data) < data_row_max:
            for j in range(len(data)+2, data_row_max+2):
                out_lines[j]+=space+' '*w
        
        space = ' '*space_num

    for line in out_lines:
        print(line)

### 與`NaN`關聯的函數

Pandas 使用 `NaN` 表示缺失的資料，由於整數列無法使用 `NaN`，因此如果整數型態的列出現缺失資料，則會被自動轉為浮點數型態。下面將布林型態的 `DataFrame` 物件傳遞給一個整數型態的 `DataFrame` 物件的 `where()` 方法。該方法將 `False` 物件的元素設定為 `NaN` ，注意其結果變成了浮點數型態，而沒有 `NaN` 的列仍然為整數型態。

In [5]:
np.random.seed(41)
df_int = pd.DataFrame(np.random.randint(0, 10, (10, 3)), columns=list("ABC"))
df_int["A"] += 10
df_nan = df_int.where(df_int > 2)
#%hide
%C df_int.dtypes; df_nan.dtypes
print
%C 4 df_int; df_nan

df_int.dtypes     df_nan.dtypes
-------------     -------------
A    int32        A      int32 
B    int32        B    float64 
C    int32        C    float64 
dtype: object     dtype: object
  df_int           df_nan     
-----------    ---------------
    A  B  C        A    B    C
0  10  3  2    0  10  3.0  NaN
1  10  1  3    1  10  NaN  3.0
2  19  7  5    2  19  7.0  5.0
3  18  3  3    3  18  3.0  3.0
4  12  6  0    4  12  6.0  NaN
5  14  6  9    5  14  6.0  9.0
6  13  8  4    6  13  8.0  4.0
7  17  6  1    7  17  6.0  NaN
8  15  2  1    8  15  NaN  NaN
9  15  3  2    9  15  3.0  NaN


`isnull()` 和 `notnull()` 用於判斷元素值是否為 `NaN` ，它們傳回全是布林值的 `DataFrame` 物件。`df.notnull()` 和 `~df.isnull()` 的結果相同，但是由於 `notnull()` 少建立一個臨時物件，其運算效率更高一些。

In [6]:
%C 4 df_nan.isnull(); df_nan.notnull()

   df_nan.isnull()          df_nan.notnull()   
----------------------    ---------------------
       A      B      C          A      B      C
0  False  False   True    0  True   True  False
1  False   True  False    1  True  False   True
2  False  False  False    2  True   True   True
3  False  False  False    3  True   True   True
4  False  False   True    4  True   True  False
5  False  False  False    5  True   True   True
6  False  False  False    6  True   True   True
7  False  False   True    7  True   True  False
8  False   True   True    8  True  False  False
9  False  False   True    9  True   True  False


`count()` 傳回每行或每列的 `非 NaN 元素` 的個數：

In [7]:
%C 4 df_nan.count(); df_nan.count(axis=1)

df_nan.count()    df_nan.count(axis=1)
--------------    --------------------
A    10           0    2              
B     8           1    2              
C     5           2    3              
dtype: int64      3    3              
                  4    2              
                  5    3              
                  6    3              
                  7    2              
                  8    1              
                  9    2              
                  dtype: int64        


對於包含 `NaN` 元素的資料，最簡單的辦法就是呼叫 `dropna()` 以刪除包含 `NaN` 的行或列，當全部使用預設參數時，將刪除包含 `NaN` 的所有行。可以透過 `thresh` 參數指定 `NaN` 個數的設定值，刪除所有 `NaN` 個數大於等於該設定值的行。

In [8]:
%C df_nan.dropna(); df_nan.dropna(thresh=2)

df_nan.dropna()     df_nan.dropna(thresh=2)
---------------     -----------------------
    A    B    C         A    B    C        
2  19  7.0  5.0     0  10  3.0  NaN        
3  18  3.0  3.0     1  10  NaN  3.0        
5  14  6.0  9.0     2  19  7.0  5.0        
6  13  8.0  4.0     3  18  3.0  3.0        
                    4  12  6.0  NaN        
                    5  14  6.0  9.0        
                    6  13  8.0  4.0        
                    7  17  6.0  NaN        
                    9  15  3.0  NaN        


當 行資料 按照某種實體順序(例如時間)排列時，可以使用 `NaN` 前後的資料對其進行填充。`ffill()` 使用之前的資料填充，而 `bfill()` 則使用之後的資料填充。`interpolate()` 使用前後資料進行內插填充：

In [9]:
%C df_nan.ffill(); df_nan.bfill(); df_nan.interpolate()

df_nan.ffill()      df_nan.bfill()      df_nan.interpolate()
---------------     ---------------     --------------------
    A    B    C         A    B    C         A    B    C     
0  10  3.0  NaN     0  10  3.0  3.0     0  10  3.0  NaN     
1  10  3.0  3.0     1  10  7.0  3.0     1  10  5.0  3.0     
2  19  7.0  5.0     2  19  7.0  5.0     2  19  7.0  5.0     
3  18  3.0  3.0     3  18  3.0  3.0     3  18  3.0  3.0     
4  12  6.0  3.0     4  12  6.0  9.0     4  12  6.0  6.0     
5  14  6.0  9.0     5  14  6.0  9.0     5  14  6.0  9.0     
6  13  8.0  4.0     6  13  8.0  4.0     6  13  8.0  4.0     
7  17  6.0  4.0     7  17  6.0  NaN     7  17  6.0  4.0     
8  15  6.0  4.0     8  15  3.0  NaN     8  15  4.5  4.0     
9  15  3.0  4.0     9  15  3.0  NaN     9  15  3.0  4.0     


`interpolate()` 預設使用等距線性內插，可以透過其 `method` 參數指定內插演算法。在下面的實例中，第 0 個元素和第 2 個元素的數值分別為 3.0 和 7.0，因此當 `method` 參數預設省時索引為 1 的 `NaN` 被填充為前後兩個元素的平均值 5.0。而當 `method` 為 `"index"` 時，則使用索引值進行內插運算。由於第 1 個元素的索引與第 2 個元素的索引接近，因此其內插結果也接近第二個元素的值。

In [11]:
s = pd.Series([3, np.NaN, 7], index=[0, 8, 9])
%C s; s.interpolate(); s.interpolate(method="index")

      s            s.interpolate()     s.interpolate(method="index")
--------------     ---------------     -----------------------------
0    3.0           0    3.0            0    3.000000                
8    NaN           8    5.0            8    6.555556                
9    7.0           9    7.0            9    7.000000                
dtype: float64     dtype: float64      dtype: float64               


此外還可以使用字典參數讓 `fillna()` 對不同的列使用不同的值填充 `NaN`：

In [12]:
print( df_nan.fillna({"B":-999, "C":0}) )

    A      B    C
0  10    3.0  0.0
1  10 -999.0  3.0
2  19    7.0  5.0
3  18    3.0  3.0
4  12    6.0  0.0
5  14    6.0  9.0
6  13    8.0  4.0
7  17    6.0  0.0
8  15 -999.0  0.0
9  15    3.0  0.0


各種聚合方法的 `skipna` 參數預設為 `True` ，因此計算時將忽略 `NaN` 元素，注意每行或每列是單獨運算的。如果需要忽略包含 `NaN` 的整行，需要先呼叫 `dropna()` 。若將 `skipna` 參數設定為 `False` ，則包含 `NaN` 的行或列的運算結果為 `NaN`。

In [13]:
%C df_nan; df_nan.sum(); df_nan.sum(skipna=False); df_nan.dropna().sum()

    df_nan           df_nan.sum()      df_nan.sum(skipna=False)     df_nan.dropna().sum()
---------------     --------------     ------------------------     ---------------------
    A    B    C     A    143.0         A    143.0                   A    64.0            
0  10  3.0  NaN     B     42.0         B      NaN                   B    24.0            
1  10  NaN  3.0     C     24.0         C      NaN                   C    21.0            
2  19  7.0  5.0     dtype: float64     dtype: float64               dtype: float64       
3  18  3.0  3.0                                                                          
4  12  6.0  NaN                                                                          
5  14  6.0  9.0                                                                          
6  13  8.0  4.0                                                                          
7  17  6.0  NaN                                                                          
8  15  NaN

`df.combine_first(other)` 使用 `other` 填充 df 中的 `NaN` 元素。它將 df 中的 `NaN` 元素取代為 `other` 中對應標籤的元素。在下面的實例中，df_nan 中索引為 1, 2, 8, 9的行中的 `NaN` 被取代為 df_other 中對應的值：

In [14]:
df_other = pd.DataFrame(np.random.randint(0, 10, (4, 2)), 
                        columns=["B", "C"], 
                        index=[1, 2, 8, 9])
# print( df_nan.combine_first(df_other) )
%C df_nan; df_other; df_nan.combine_first(df_other)

    df_nan          df_other     df_nan.combine_first(df_other)
---------------     --------     ------------------------------
    A    B    C        B  C          A    B    C               
0  10  3.0  NaN     1  4  1      0  10  3.0  NaN               
1  10  NaN  3.0     2  2  3      1  10  4.0  3.0               
2  19  7.0  5.0     8  4  5      2  19  7.0  5.0               
3  18  3.0  3.0     9  4  5      3  18  3.0  3.0               
4  12  6.0  NaN                  4  12  6.0  NaN               
5  14  6.0  9.0                  5  14  6.0  9.0               
6  13  8.0  4.0                  6  13  8.0  4.0               
7  17  6.0  NaN                  7  17  6.0  NaN               
8  15  NaN  NaN                  8  15  4.0  5.0               
9  15  3.0  NaN                  9  15  3.0  5.0               
